# LLM Fine-tuning with Reinforcement Learning (RLHF)
## Using Unsloth for Efficient Training

This notebook demonstrates:
1. **Supervised Fine-tuning (SFT)** - Training on instruction-response pairs
2. **Reinforcement Learning from Human Feedback (RLHF)** - Training with preferred/rejected outputs

### What We'll Cover:
- Loading pre-trained models with Unsloth
- Chat templates and formatting
- Dataset preparation for RLHF
- DPO (Direct Preference Optimization) training
- Model evaluation and inference

## 1. Setup and Installation

First, install required packages. Unsloth provides 2x faster training with less memory usage.

In [11]:
# Install Unsloth and dependencies
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install datasets transformers

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-rwtzj7_s/unsloth_aa9394de82154bbeb43bcdbef6a5e6b7
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-rwtzj7_s/unsloth_aa9394de82154bbeb43bcdbef6a5e6b7
  Resolved https://github.com/unslothai/unsloth.git to commit aa7cfa1d3cd8f3fa7c0423ee227a7748c4228630
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 9.5 MB/s eta 0:00:00


In [12]:
import torch
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import load_dataset
from trl import DPOTrainer, DPOConfig  # Added DPOConfig
from transformers import TrainingArguments, TextStreamer
import json

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Check TRL version
import trl
print(f"TRL version: {trl.__version__}")

PyTorch version: 2.8.0+cu126
CUDA available: True
GPU: Tesla T4
TRL version: 0.24.0


## 2. Load Pre-trained Model

We'll use **Llama 3.2 1B** model - a small, efficient model perfect for fine-tuning.

### Key Parameters:
- `max_seq_length`: Maximum sequence length (2048 tokens)
- `dtype`: Use bfloat16 for faster training if supported
- `load_in_4bit`: Enable 4-bit quantization to reduce memory usage
- `r`: LoRA rank (16) - controls adapter capacity
- `lora_alpha`: LoRA scaling factor (16)

In [4]:
# Model configuration
max_seq_length = 2048
dtype = None  # Auto-detect. Use Float16 for Tesla T4, V100, or bfloat16 for Ampere+
load_in_4bit = True  # Use 4bit quantization to reduce memory usage

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

print("✓ Model loaded successfully!")
print(f"Model name: {model.config._name_or_path}")
print(f"Vocabulary size: {len(tokenizer)}")

==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

✓ Model loaded successfully!
Model name: unsloth/llama-3.2-1b-instruct-unsloth-bnb-4bit
Vocabulary size: 128256


## 3. Configure LoRA Adapters

**LoRA (Low-Rank Adaptation)** allows efficient fine-tuning by training small adapter layers instead of the entire model.

### Target Modules:
- `q_proj`, `k_proj`, `v_proj`: Attention query, key, value projections
- `o_proj`: Attention output projection
- `gate_proj`, `up_proj`, `down_proj`: MLP layers

In [5]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,  # Optimized to 0 for faster training
    bias="none",
    use_gradient_checkpointing="unsloth",  # Long context support
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

print("✓ LoRA adapters configured!")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

Unsloth 2025.11.3 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


✓ LoRA adapters configured!
Trainable parameters: 11,272,192
Total parameters: 785,713,152


## 4. Understanding Chat Templates

Chat templates format conversations for the model. Different models use different formats.

### Llama 3 Format:
```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{system_message}<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{user_message}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
{assistant_response}<|eot_id|>
```

In [6]:
# Example: Format a conversation using the chat template
messages = [
    {"role": "system", "content": "You are a helpful coding assistant."},
    {"role": "user", "content": "Write a Python function to calculate fibonacci numbers."},
]

formatted = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

print("Formatted prompt:")
print(formatted)
print("\nSpecial tokens:")
print(f"BOS token: {tokenizer.bos_token}")
print(f"EOS token: {tokenizer.eos_token}")
print(f"PAD token: {tokenizer.pad_token}")

Formatted prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 10 Nov 2025

You are a helpful coding assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a Python function to calculate fibonacci numbers.<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Special tokens:
BOS token: <|begin_of_text|>
EOS token: <|eot_id|>
PAD token: <|finetune_right_pad_id|>


## 5. Prepare Dataset for RLHF

For **DPO (Direct Preference Optimization)**, we need a dataset with:
- `prompt`: The input question/instruction
- `chosen`: The preferred response (high quality)
- `rejected`: The rejected response (lower quality)

We'll use the **Anthropic HH-RLHF** dataset - a collection of helpful and harmless conversations.

In [7]:
# Load RLHF dataset
dataset = load_dataset("Anthropic/hh-rlhf", split="train[:1000]")  # Use 1000 samples for demo

print(f"Dataset size: {len(dataset)}")
print("\nFirst example:")
print(json.dumps(dataset[0], indent=2)[:500] + "...")

README.md: 0.00B [00:00, ?B/s]

harmless-base/train.jsonl.gz:   0%|          | 0.00/13.2M [00:00<?, ?B/s]

helpful-base/train.jsonl.gz:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

helpful-online/train.jsonl.gz:   0%|          | 0.00/20.1M [00:00<?, ?B/s]

helpful-rejection-sampled/train.jsonl.gz:   0%|          | 0.00/25.7M [00:00<?, ?B/s]

...

helpful-online/test.jsonl.gz:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

helpful-rejection-sampled/test.jsonl.gz:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/160800 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8552 [00:00<?, ? examples/s]

Dataset size: 1000

First example:
{
  "chosen": "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here\u2019s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama\u201...


In [8]:
# Format dataset for DPO training
def format_dpo_dataset(example):
    """
    Convert raw dataset into DPO format.

    Input format:
    - chosen: "\n\nHuman: {question}\n\nAssistant: {good_answer}"
    - rejected: "\n\nHuman: {question}\n\nAssistant: {bad_answer}"

    Output format:
    - prompt: The user's question
    - chosen: The preferred assistant response
    - rejected: The rejected assistant response
    """
    # Extract prompt from chosen response (same for both)
    chosen_parts = example["chosen"].split("\n\nAssistant: ")
    rejected_parts = example["rejected"].split("\n\nAssistant: ")

    # Get the human's question
    prompt = chosen_parts[0].replace("\n\nHuman: ", "").strip()

    # Get responses
    chosen_response = chosen_parts[1].strip() if len(chosen_parts) > 1 else ""
    rejected_response = rejected_parts[1].strip() if len(rejected_parts) > 1 else ""

    return {
        "prompt": prompt,
        "chosen": chosen_response,
        "rejected": rejected_response
    }

# Apply formatting
dpo_dataset = dataset.map(format_dpo_dataset, remove_columns=dataset.column_names)

print("✓ Dataset formatted for DPO!")
print("\nExample:")
print(f"Prompt: {dpo_dataset[0]['prompt'][:100]}...")
print(f"Chosen: {dpo_dataset[0]['chosen'][:100]}...")
print(f"Rejected: {dpo_dataset[0]['rejected'][:100]}...")

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

✓ Dataset formatted for DPO!

Example:
Prompt: What are some cuss words in english?...
Chosen: Here’s an incomplete list.

Ass, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scr...
Rejected: Here’s an incomplete list.

Ass, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scr...


## 6. DPO Training Configuration

**DPO (Direct Preference Optimization)** trains the model to prefer chosen responses over rejected ones.

### How DPO Works:
1. Model generates probability for both chosen and rejected responses
2. Loss function maximizes the difference in log-probabilities
3. Model learns to assign higher probability to preferred responses

### Key Hyperparameters:
- `beta`: Temperature parameter for DPO (0.1) - controls strength of preference
- `learning_rate`: 5e-5 - small LR for stable training
- `num_train_epochs`: 1 - one pass through data

In [13]:
# Training arguments using DPOConfig (NOT TrainingArguments)
training_args = DPOConfig(
    output_dir="./dpo_output",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=5e-5,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    warmup_steps=50,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    seed=3407,
    report_to="none",
    beta=0.1,  # DPO temperature - NOW IN CONFIG
    max_length=max_seq_length,  # NOW IN CONFIG
    max_prompt_length=512,  # NOW IN CONFIG
)

print("✓ Training arguments configured!")
print(f"Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"Total training steps: {len(dpo_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)}")

✓ Training arguments configured!
Effective batch size: 8
Total training steps: 125


In [14]:
# Initialize DPO Trainer (simplified)
dpo_trainer = DPOTrainer(
    model=model,
    args=training_args,  # DPOConfig contains all parameters
    train_dataset=dpo_dataset,
    tokenizer=tokenizer,
)

print("✓ DPO Trainer initialized!")

Extracting prompt in train dataset (num_proc=6):   0%|          | 0/1000 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=6):   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=6):   0%|          | 0/1000 [00:00<?, ? examples/s]

✓ DPO Trainer initialized!


## 7. Train the Model

Now we'll train the model using DPO. This will take several minutes depending on your GPU.

**What happens during training:**
- Model processes prompt + chosen response pairs
- Model processes prompt + rejected response pairs
- Loss encourages model to prefer chosen over rejected
- LoRA adapters are updated, base model stays frozen

In [15]:
# Start training
print("Starting DPO training...\n")
dpo_trainer.train()
print("\n✓ Training complete!")

The model is already on multiple devices. Skipping the move to device specified in `args`.


Starting DPO training...



==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 125
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 11,272,192 of 1,247,086,592 (0.90% trained)


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss
10,0.692900,-0.000391,-0.000947,0.162500,0.000556,-157.123749,-158.937469,1.820835,1.807598,0,0,0
20,0.692600,0.000580,-0.000534,0.200000,0.001115,-150.054901,-171.484222,1.895143,1.897715,No Log,No Log,No Log
30,0.692700,0.003051,0.002038,0.137500,0.001012,-152.065582,-165.793747,1.837286,1.847338,No Log,No Log,No Log
40,0.689400,0.008751,0.001084,0.237500,0.007667,-144.079193,-155.743118,1.819143,1.827328,No Log,No Log,No Log
50,0.686100,0.015324,0.000327,0.237500,0.014997,-162.985260,-170.415802,1.900355,1.853308,No Log,No Log,No Log
60,0.687100,-0.003653,-0.017731,0.250000,0.014077,-161.512146,-174.441925,1.885795,1.878795,No Log,No Log,No Log
70,0.674700,-0.011327,-0.053191,0.225000,0.041864,-166.971329,-185.791870,1.877349,1.874434,No Log,No Log,No Log
80,0.670500,-0.034810,-0.093996,0.237500,0.059186,-152.828751,-173.139023,1.850101,1.861604,No Log,No Log,No Log
90,0.674600,-0.081069,-0.127187,0.300000,0.046118,-151.416656,-165.435455,1.836901,1.882297,No Log,No Log,No Log
100,0.665400,-0.099211,-0.180230,0.187500,0.081019,-136.087280,-149.379517,1.839523,1.809632,No Log,No Log,No Log


Unsloth: Will smartly offload gradients to save VRAM!

✓ Training complete!


## 8. Save the Fine-tuned Model

We'll save both:
1. **LoRA adapters** - Small, efficient (few MB)
2. **Merged model** - Full model with adapters merged (optional)

In [16]:
# Save LoRA adapters
model.save_pretrained("dpo_lora_model")
tokenizer.save_pretrained("dpo_lora_model")
print("✓ LoRA adapters saved to 'dpo_lora_model/'")

# Optional: Save merged model (16bit for better quality)
# model.save_pretrained_merged("dpo_merged_model", tokenizer, save_method="merged_16bit")
# print("✓ Merged model saved to 'dpo_merged_model/'")

✓ LoRA adapters saved to 'dpo_lora_model/'


## 9. Test the Fine-tuned Model

Let's test our fine-tuned model with some prompts to see how it performs!

In [17]:
# Enable inference mode
FastLanguageModel.for_inference(model)

# Test prompt
test_messages = [
    {"role": "system", "content": "You are a helpful, respectful and honest assistant."},
    {"role": "user", "content": "What are some tips for staying healthy?"}
]

inputs = tokenizer.apply_chat_template(
    test_messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")

# Generate response with streaming
text_streamer = TextStreamer(tokenizer, skip_prompt=True)
print("🤖 Assistant: ", end="")

outputs = model.generate(
    input_ids=inputs,
    streamer=text_streamer,
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


🤖 Assistant: Staying healthy is a wonderful goal. Here are some valuable tips to help you achieve it:

1. **Get Moving**: Regular exercise can help keep your heart rate up, improve sleep, and boost mood. Aim for at least 30 minutes of moderate-intensity physical activity per day.
2. **Eat a Balanced Diet**: Focus on whole, unprocessed foods like fruits, vegetables, whole grains, lean proteins, and healthy fats. Aim to include a variety of colors on your plate to ensure you're getting a range of nutrients.
3. **Stay Hydrated**: Drink plenty of water throughout the day, aiming for at least eight cups (64 ounces) daily. Limit sugary drinks and caffeine to avoid dehydration.
4. **Get Enough Sleep**: Aim for 7-9 hours of sleep each night to help your body and mind recharge. Establish a consistent sleep schedule and create a relaxing bedtime routine.
5. **Manage Stress**: Engage in stress-reducing activities like meditation, yoga, or deep breathing exercises. Aim for at least 10 minutes of r